In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from biolearn.data_library import DataLibrary
from scipy.stats import pearsonr
import math

In [4]:
%%time 
wb_='data/array/GSE41169_series_matrix.txt'
wb=pd.read_table(wb_,skiprows=32+14+26,index_col=0).drop('!series_matrix_table_end').round(3)
wb_t=wb.T

CPU times: user 7.24 s, sys: 725 ms, total: 7.96 s
Wall time: 8.11 s


In [6]:
%%time 
age=pd.read_table(wb_,skiprows=32+14,nrows=0,index_col=0).columns.str.strip('age: ').astype(float).tolist()

CPU times: user 32.5 ms, sys: 8.13 ms, total: 40.6 ms
Wall time: 37.7 ms


In [8]:
wb_t['age']=age

In [ ]:
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

In [6]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/data/ref/HM450.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man['cg']=man.index
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')
#man['bin']=man['cor']>0

In [9]:
man['cor'].rename('cor_450').to_csv('data/res/cor450.csv')

In [5]:
hmr=pd.read_table('/labs/mpsnyder/moqri/epi_deconv/data/neo/neo.hmr',header=None,index_col=3,usecols=[0,1,2,3])
hmr['b']=hmr[0].str[3:].astype(int)*10**9+hmr[1]
hmr['e']=hmr[0].str[3:].astype(int)*10**9+hmr[2]
hmr=hmr.sort_values('b')

In [ ]:
pv=.2631
cg=wb_cor[wb_cor>pv].head(1).index[0]
pearsonr(wb_t[cg],wb_t['age'])

In [ ]:
n=len(wb_t)
r=.17
(n-2)**(1/2)*r/(1-r**2)**(1/2)

In [ ]:
%%time 
#man=man[man['l']>0].copy()
cont=[]
c=0
last=False
last_l=-1
for i,b in man[['l','cor']].iterrows():
    if (b['cor']>pv):
        #if (b['l']>last_l or last==False):
        if (last==False):
            c+=1
            #last_l=b['l']
        cont.append(c)
        last=True  
    else:
        cont.append(-1)
        last=False

In [51]:
man['cont']=cont
mgc=pd.DataFrame(man.groupby('cont')[0].count())
mgc['chr']=man.groupby('cont')[0].first()
mgc['b']=man.groupby('cont')[1].first()
mgc['e']=man.groupby('cont')[1].last()
mgc['cor']=man.groupby('cont')['cor'].mean()
topl=mgc[mgc[0]>=3].sort_values('cor')
topl.tail(20)

,0,chr,b,e,cor
cont,,,,,
3227,3,chr4,121381109,121381120,0.544754
2001,6,chr2,218872669,218874009,0.552954
10819,9,chr17,44004441,44005023,0.553418
10936,3,chr17,58755629,58755681,0.554242
9624,3,chr15,31483202,31484246,0.555696
11729,3,chr19,38869672,38869824,0.568889
530,8,chr1,68046855,68047379,0.569120
3272,3,chr4,146636936,146637282,0.572801
5815,3,chr7,130733487,130733721,0.576406


In [33]:
#man=man[man['cor']>.170].copy()

In [6]:
%%time
l=[]
cur=0
for i in range(len(man)):
    if cur>=len(hmr):
        l.append(-1)
    else: 
        pos=man['pos'].iloc[i]
        cg=man['cg'].iloc[i]
        b=hmr['b'].iloc[cur]
        e=hmr['e'].iloc[cur]        
        if pos<b:
            l.append(-1)
        if pos>=b and pos<=e:
            l.append(cur)
        if pos>e:
            while (pos>hmr['e'].iloc[cur] and cur<len(hmr)-1):
                cur+=1
                b=hmr['b'].iloc[cur]
                e=hmr['e'].iloc[cur] 
            if pos>=b and pos<=e:
                l.append(cur)
            else:
                l.append(-1)

CPU times: user 26.5 s, sys: 13 ms, total: 26.5 s
Wall time: 26.5 s


In [7]:
man['l']=l

In [8]:
mg=pd.DataFrame(man.groupby('l')[0].count())
mg=mg[mg[0]>=4].copy()
print(len(mg))
mg['cor']=man.groupby('l')['cor'].mean()

16721


In [23]:
mg.columns=['l','cor']
hmr=hmr.reset_index().merge(mg,left_index=True,right_index=True).sort_values('cor').copy()

In [36]:
hmr.tail(20)

,3,0,1,2,b,e,l,cor
31701,HYPO14036,chr14,103207142,103208523,14103207142,14103208523,5,0.439890
36078,HYPO18413,chr17,6775822,6776462,17006775822,17006776462,12,0.448217
36634,HYPO18969,chr17,28985870,28986481,17028985870,17028986481,6,0.449117
19194,HYPO43087,chr8,56447897,56448292,8056447897,8056448292,5,0.449230
14728,HYPO38621,chr6,42177967,42178478,6042177967,6042178478,4,0.455844
42161,HYPO24496,chr19,51887336,51888227,19051887336,19051888227,12,0.459930
41322,HYPO23657,chr19,38869276,38870374,19038869276,19038870374,4,0.467760
41224,HYPO23559,chr19,36244899,36246369,19036244899,19036246369,4,0.471813
42406,HYPO24741,chr19,57708673,57709644,19057708673,19057709644,11,0.475737
12403,HYPO36296,chr5,93620363,93621878,5093620363,5093621878,11,0.477521


In [ ]:
%%time
mono_='data/cell/GSE56046_methylome_normalized.txt'
mono=pd.read_table(mono_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,mono).round(3)
mono=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56046/matrix/GSE56046_series_matrix.txt.gz',skiprows=46,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
mono_t=mono.T
mono_t['age']=age.head(100).values
mono_cor=mono_t.corrwith(mono_t['age']).drop('age').sort_values()

In [ ]:
%%time
t_='data/cell/GSE56581_methylome_normalized.txt'
t=pd.read_table(t_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,t).round(3)
t=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56581/matrix/GSE56581_series_matrix.txt.gz',skiprows=40,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
t_t=t.T
t_t['age']=age.head(100).values
t_cor=t_t.corrwith(t_t['age']).drop('age').sort_values()

In [ ]:
%%time
liver_='data/cell/GSE48325_series_matrix.txt'
liver=pd.read_table(liver_,skiprows=27+33,index_col=0).round(3)
age=pd.read_table(liver_,skiprows=27,nrows=50,index_col=0).iloc[8].str.strip('age: ').astype(int)
liver_t=liver.T
liver_t['age']=age.values
liver_cor=liver_t.corrwith(liver_t['age']).drop('age').sort_values()

In [ ]:
dd=pd.concat([wb_cor,t_cor,mono_cor,liver_cor],axis=1).drop('!series_matrix_table_end')
dd.corr()

In [ ]:
t_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/cd4.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]
m_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/mono.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]